In [298]:
import pandas as pd

file_1 = pd.read_csv('C:/Users/zzaer/OneDrive/바탕 화면/2025 채린/아러다/train.csv')
file_2 = pd.read_csv('C:/Users/zzaer/OneDrive/바탕 화면/2025 채린/아러다/test.csv')
LABEL_COL = "fraud"   # 정답 컬럼
ID_COL    = "id"      # ID   컬럼

In [299]:
# 결측치 처리

In [300]:
# device_distinct_emails_8w
# 크게 영향을 끼치지 않을 정도로 결측치 값이 너무 적음. 
# 주로 1과 2 중에서 최빈값인 1로 처리함.

file_1['device_distinct_emails_8w'] = file_1['device_distinct_emails_8w'].fillna(1) # 최빈값인 1로 처리
file_2['device_distinct_emails_8w'] = file_2['device_distinct_emails_8w'].fillna(1) # 최빈값인 1로 처리

In [301]:
# session_length_in_minutes
# 평균값으로 대체 

mean_val = file_1['session_length_in_minutes'].mean()
file_1['session_length_in_minutes'] = file_1['session_length_in_minutes'].fillna(mean_val)

mean_val = file_2['session_length_in_minutes'].mean()
file_2['session_length_in_minutes'] = file_2['session_length_in_minutes'].fillna(mean_val)

In [302]:
# current_address_months_count  
# 평균균값으로 대체

mean_val = file_1['current_address_months_count'].mean()
file_1['current_address_months_count'] = file_1['current_address_months_count'].fillna(mean_val)

mean_val = file_2['current_address_months_count'].mean()
file_2['current_address_months_count'] = file_2['current_address_months_count'].fillna(mean_val)

In [303]:
# bank_months_count
# 사기인 경우와 사기가 아닌 경우를 나누어, 평균, 표준편차를 구하여 조건부 랜덤 샘플링 기반 결측 대체 방벙블 사용함 

fraud_mean = file_1[file_1['fraud'] == 1]['bank_months_count'].mean()
nonfraud_mean = file_1[file_1['fraud'] == 0]['bank_months_count'].mean()

print(f"사기인 경우 평균: {fraud_mean:.2f}")
print(f"사기가 아닌 경우 평균: {nonfraud_mean:.2f}")
fraud_stats = file_1[file_1['fraud'] == 1]['bank_months_count'].agg(['mean', 'std'])
nonfraud_stats = file_1[file_1['fraud'] == 0]['bank_months_count'].agg(['mean', 'std'])

print(f"사기인 경우 → 평균: {fraud_stats['mean']:.2f}, 표준편차: {fraud_stats['std']:.2f}")
print(f"사기 아닌 경우 → 평균: {nonfraud_stats['mean']:.2f}, 표준편차: {nonfraud_stats['std']:.2f}")

사기인 경우 평균: 17.33
사기가 아닌 경우 평균: 14.84
사기인 경우 → 평균: 17.33, 표준편차: 11.84
사기 아닌 경우 → 평균: 14.84, 표준편차: 11.52


In [304]:
import numpy as np

cond_missing = file_1['bank_months_count'].isna()

# fraud==1 기준 평균±표준편차
fraud_mean = 17.33
fraud_std = 11.84

# fraud==0 기준 평균±표준편차
nonfraud_mean = 14.84
nonfraud_std = 11.52

# 랜덤 생성 함수
def sample_from_normal(mean, std, size):
    return np.clip(np.random.normal(mean, std, size), 0, None)

# 사기인 경우
fraud_mask = cond_missing & (file_1['fraud'] == 1)
file_1.loc[fraud_mask, 'bank_months_count'] = sample_from_normal(fraud_mean, fraud_std, fraud_mask.sum())

# 사기 아닌 경우
nonfraud_mask = cond_missing & (file_1['fraud'] == 0)
file_1.loc[nonfraud_mask, 'bank_months_count'] = sample_from_normal(nonfraud_mean, nonfraud_std, nonfraud_mask.sum())

In [305]:
import numpy as np

# train에서 구한 평균과 표준편차
fraud_mean = 17.33
fraud_std = 11.84

nonfraud_mean = 14.84
nonfraud_std = 11.52

# train에서 사기 비율
fraud_ratio = file_1['fraud'].mean()
nonfraud_ratio = 1 - fraud_ratio

# 전체 가중 평균과 표준편차
overall_mean = fraud_mean * fraud_ratio + nonfraud_mean * nonfraud_ratio
overall_std = fraud_std * fraud_ratio + nonfraud_std * nonfraud_ratio

# test 결측치 채우기
def sample_from_normal(mean, std, size):
    return np.clip(np.random.normal(mean, std, size), 0, None)

cond_missing_test = file_2['bank_months_count'].isna()
file_2.loc[cond_missing_test, 'bank_months_count'] = sample_from_normal(overall_mean, overall_std, cond_missing_test.sum())


In [306]:
# initial_transfer_amount
# 결측치 너무 많아서 그냥 제거

file_1 = file_1.drop(columns=['initial_transfer_amount'])
file_2 = file_2.drop(columns=['initial_transfer_amount'])

In [307]:
# prev_address_months_count
# 결측치 너무 많아서 그냥 제거 

file_1 = file_1.drop(columns=['prev_address_months_count'])
file_2 = file_2.drop(columns=['prev_address_months_count'])

In [308]:
file_1.isnull().sum()

id                                  0
bank_months_count                   0
branch_application_count_8weeks     0
credit_risk_score                   0
current_address_months_count        0
customer_age                        0
date_of_birth_distinct_emails_4w    0
days_since_request                  0
device_distinct_emails_8w           0
device_os                           0
device_previously_fraudulent        0
email_is_free                       0
employment_status                   0
foreign_request                     0
has_other_cards                     0
housing_status                      0
income                              0
keep_alive_session                  0
month                               0
name_email_similarity               0
payment_type                        0
phone_home_valid                    0
phone_mobile_valid                  0
requested_credit_limit              0
session_length_in_minutes           0
source                              0
velocity_24h

In [309]:
file_2.isnull().sum()

id                                  0
bank_months_count                   0
branch_application_count_8weeks     0
credit_risk_score                   0
current_address_months_count        0
customer_age                        0
date_of_birth_distinct_emails_4w    0
days_since_request                  0
device_distinct_emails_8w           0
device_os                           0
device_previously_fraudulent        0
email_is_free                       0
employment_status                   0
foreign_request                     0
has_other_cards                     0
housing_status                      0
income                              0
keep_alive_session                  0
month                               0
name_email_similarity               0
payment_type                        0
phone_home_valid                    0
phone_mobile_valid                  0
requested_credit_limit              0
session_length_in_minutes           0
source                              0
velocity_24h

In [310]:
file_2.head()

,id,bank_months_count,branch_application_count_8weeks,credit_risk_score,current_address_months_count,customer_age,date_of_birth_distinct_emails_4w,days_since_request,device_distinct_emails_8w,device_os,...,payment_type,phone_home_valid,phone_mobile_valid,requested_credit_limit,session_length_in_minutes,source,velocity_24h,velocity_4w,velocity_6h,zip_count_4w
0,0,26.000000,11.0,25,81.0,30,12,10.05,1.0,other,...,AA,False,True,200.0,0.9,INTERNET,6297.90,5647.48,13882.70,1161
1,1,8.802071,0.0,138,11.0,50,5,0.02,1.0,windows,...,AC,True,True,500.0,7.8,INTERNET,4282.20,5369.74,6667.63,2236
2,2,28.000000,1883.0,162,15.0,10,4,0.01,1.0,linux,...,AA,False,True,200.0,29.0,INTERNET,8751.22,5048.42,7576.66,2450
3,3,30.000000,29.0,58,134.0,40,4,0.02,1.0,windows,...,AB,True,True,200.0,2.3,INTERNET,4074.20,6697.37,1863.00,1733
4,4,28.000000,10.0,65,3.0,20,6,0.02,1.0,other,...,AA,False,True,200.0,8.2,INTERNET,5118.08,4321.08,10149.49,227


In [311]:
# One-Hot Encoding 대상 (drop_first=True로 다중공선성 방지)
onehot_cols = ['device_os', 'employment_status', 'housing_status', 'payment_type']

# Binary Encoding 대상
binary_map = {'INTERNET': 0, 'TELEAPP': 1}

# 1. One-Hot 인코딩
file_1 = pd.get_dummies(file_1, columns=onehot_cols, drop_first=True)

# 2. Binary 인코딩 (source)
file_1['source'] = file_1['source'].map(binary_map)

# 3. Boolean 컬럼은 이미 0/1 형태이므로 변환 불필요
# 단, 필요시 int로 명확히 변환하고 싶다면 아래처럼도 가능
bool_cols = ['email_is_free', 'foreign_request', 'has_other_cards',
             'keep_alive_session', 'phone_home_valid', 'phone_mobile_valid']

file_1[bool_cols] = file_1[bool_cols].astype(int)


In [312]:
# One-Hot Encoding 대상 (drop_first=True로 다중공선성 방지)
onehot_cols = ['device_os', 'employment_status', 'housing_status', 'payment_type']

# Binary Encoding 대상
binary_map = {'INTERNET': 0, 'TELEAPP': 1}

# 1. One-Hot 인코딩 (train과 test 모두 동일하게 처리)
file_2 = pd.get_dummies(file_2, columns=onehot_cols, drop_first=True)

# 2. Binary 인코딩 (source)
file_2['source'] = file_2['source'].map(binary_map)

# 3. Boolean 컬럼은 이미 0/1 형태이므로 변환 불필요
# 단, 필요시 int로 명확히 변환하고 싶다면 아래처럼도 가능
bool_cols = ['email_is_free', 'foreign_request', 'has_other_cards',
             'keep_alive_session', 'phone_home_valid', 'phone_mobile_valid']

file_2[bool_cols] = file_2[bool_cols].astype(int)


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
X = file_1.drop("fraud", axis=1)
y = file_1["fraud"]

X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42,
    n_jobs=-1
)

model.fit(X_tr, y_tr)

val_probs = model.predict_proba(X_val)[:, 1]
val_ap = average_precision_score(y_val, val_probs)
print(f"Validation Average Precision : {val_ap:.6f}")


[LightGBM] [Info] Number of positive: 6216, number of negative: 553784
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.046892 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3196
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011100 -> initscore=-4.489648
[LightGBM] [Info] Start training from score -4.489648
Validation Average Precision : 0.164255


In [316]:

model.fit(X, y)

test_probs = model.predict_proba(file_2)[:, 1]
submission = pd.read_csv("sample_submission.csv")
submission[LABEL_COL] = test_probs
submission.to_csv("C:/Users/zzaer/OneDrive/바탕 화면/2025 채린/아러다/submission.csv", index=False)
print("submission.csv 저장 완료")


[LightGBM] [Info] Number of positive: 7770, number of negative: 692230
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3197
[LightGBM] [Info] Number of data points in the train set: 700000, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011100 -> initscore=-4.489648
[LightGBM] [Info] Start training from score -4.489648
submission.csv 저장 완료
